# Demo

###### Imports

In [1]:
# This Python file uses the following encoding: utf-8
from lib.predict_pdf import predict_documents_pdf, predict_documents_str_we, predictions_dataframe, load_prediction_models, predict_documents_pdf_we, load_models, load_models_we
from lib import predict_pdf as pp
from lib.utils import move_flattened_files
from functools import partial
from lib.utils import download_from_storage_if_not_present
from lib import extract_parts as ep
import pandas as pd
import json
import os

Using TensorFlow backend.


###### Retrieval of test files from GCS

In [2]:
def get_test_files():
    import subprocess
    %mkdir files_to_predict
    subprocess.call(["gsutil", "-m", "cp", "gs://infocamere/test_files/*", "./files_to_predict"])
    
get_test_files()

mkdir: cannot create directory ‘files_to_predict’: File exists


###### Retrieval of models from GCS

In [3]:
# Download resources if not found
with open("gs_resource_map.json") as f:
    gs_map = json.load(f)

for k,v in gs_map.items():
    download_from_storage_if_not_present("injenia-ricerca", v, k)

###### Load models

In [4]:
model_we = {'keras_model_file':'../models/keras_model_word_embedding.json',
            'keras_weights_file':'../models/keras_weights_word_embedding.h5',
            'reduced_dictionary_file':'../dictionaries/reduced_dictionary_cost.json'}

prediction_fn = pp.predict_document_str_we
loaded_models = load_models_we(**model_we)
predict_pdfs = partial(predict_documents_pdf_we, **loaded_models)

### Prediction

In [5]:
pdf_folder = 'files_to_predict'
pdf_names = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.lower().endswith("pdf")]
predictions, filtered_filenames = predict_pdfs(pdf_names, do_ocr=False)

##### Show predictions

In [6]:
def basenames(files):
    return [os.path.basename(f) for f in files]

In [8]:
csv_out_file = "out.csv"
df = predictions_dataframe(basenames(pdf_names), basenames(filtered_filenames), predictions, csv_out_file)

In [9]:
df.sort_values("Nome file")

,Nome file,Errore,Messaggio errore,Output rete,Predizione
1,C1.pdf,No,,0.991471,COSTITUTIVO
4,C2.pdf,No,,0.987761,COSTITUTIVO
7,C3.pdf,No,,0.987450,COSTITUTIVO
9,C4.pdf,No,,0.989425,COSTITUTIVO
5,C5.pdf,No,,0.919796,COSTITUTIVO
6,NC1.pdf,No,,0.001568,NON COSTITUTIVO
3,NC2.pdf,No,,0.002875,NON COSTITUTIVO
2,NC3.pdf,No,,0.003167,NON COSTITUTIVO
8,NC4.pdf,No,,0.002639,NON COSTITUTIVO
0,NC5.pdf,No,,0.002939,NON COSTITUTIVO


### Sentence Extraction

In [10]:
extraction_models = {
    'keras_model_filename':'../models/extraction_model_30_all.json',
    'keras_weights_filename':'../models/extraction_weights_30_all.h5',
    'reduced_dict_filename':'../dictionaries/first_5000_words_extraction.json'
}

pe = ep.PartsExtraction.load_from_files(**extraction_models)

In [11]:
data = pe.extract_parts_pdf(pdf_names[1])

In [12]:
pd.options.display.max_rows = 999
extracted = pd.DataFrame(data, columns=["frase", "classe", "poteri", "assemblea", "clausola", "non_riconducibile"])
extracted

,frase,classe,poteri,assemblea,clausola,non_riconducibile
0,Repertorio n 15924 Raccolta n 6830 ATTO COSTIT...,non_riconducibile,0.000110,0.000055,0.000142,0.999693
1,"Avanti a me Andrea Guglielmoni, Notaio in Rap...",non_riconducibile,0.000186,0.000082,0.000272,0.999460
2,Si precisa che l'autentica delle sottoscrizio...,non_riconducibile,0.000239,0.000049,0.000025,0.999687
3,"L'avvocato Riccardo Massimilla, nella citata ...",non_riconducibile,0.011209,0.001422,0.003478,0.983890
4,"Detto comparente, della cui identità personal...",non_riconducibile,0.000075,0.000043,0.000073,0.999810
5,ai fini dell'iscrizione del presente atto pre...,non_riconducibile,0.000142,0.000028,0.000014,0.999816
6,- durata: 31 dicembre 2080,non_riconducibile,0.005559,0.003538,0.001851,0.989052
7,- oggetto: la realizzazione e commercializzaz...,non_riconducibile,0.000073,0.000033,0.000029,0.999865
8,le prestazioni di servizi e di consulenza nel...,non_riconducibile,0.000069,0.000039,0.000049,0.999843
9,Tali attività dovranno essere svolte alle con...,poteri,0.977823,0.000277,0.001860,0.020040


In [13]:
pd.options.display.max_rows = 60

##### Show full text

In [14]:
for i,f in enumerate(extracted["frase"]):
    print i, f

0 Repertorio n 15924 Raccolta n 6830 ATTO COSTITUTIVO DI SOCIETA' A RESPONSABILITA' LIMITATA CON UNICO SOCIO REPUBBLICA ITALIANA L'anno duemiladiciassette, il giorno tre del mese di febbraio, in Genova, via Venti Settembre civico tre interno sedici
1  Avanti a me Andrea Guglielmoni, Notaio in Rapallo, con studio in via Lamarmora civico dieci interno uno, iscritto presso il Collegio Notarile dei distretti riuniti di Genova e Chiavari, è comparso: - Massimilla Riccardo, nato a Genova (GE) il 22 giugno 1981, domiciliato in Genova (GE), via Fieschi n 3/13, il quale dichiara di intervenire al presente atto non in proprio, ma nella sua qualità di procuratore speciale della Società "GLOBAL SKI-PIC LTD", con sede in 20 Galagalei Hplada, Herzelia (Israele), capitale sociale 100 000 USD, numero di registrazione 515145639, società di nazionalità israeliana e costituita in Israele, tale nominato con poteri a quanto infra in forza di procura speciale autenticata nelle firme dal Notaio Amos Bar-Nath